In [2]:
import searchep;

# TODO
- revise the whole initial training procedure (create a list of steps and implement them)
- get the pipeline with a GPR model working

Problems:
- translating the procedure into code that uses searchep

# Sample Hamiltonian

In [35]:
import numpy as np

def H5(kappa: complex):
    H = np.zeros((5, 5), dtype=complex)
    # 2×2 “EP block” with eigenvalues +- kappa
    H[0, 1] = 1
    H[1, 0] = kappa
    
    H[2, 2] = 2.0
    H[3, 3] = 3.0
    H[4, 4] = 4.0
    return H


### Starting from an estimate kappa_0 and sampling an orbit around that point in kappa-space

In [4]:
kappa0 = 1j
rho = 1
M = 100

def sample_orbit(center: complex, radius: float, M:int):
    '''
    center: initial kappa
    radius: radius of the orbit
    M: the number of points
    '''

    orbit = np.linspace(0, 2*np.pi, M, endpoint=False)
    kappas = center + radius * np.exp(1j*orbit)

    return kappas, orbit

kappas, orbit = sample_orbit(kappa0, rho, M)
print(f'kappas: {kappas[:5]}, \n\norbit: {orbit[:5]}')
# {orbit, 5, kappas.real, kappas.imag} 

kappas: [1.        +1.j         0.99802673+1.06279052j 0.9921147 +1.12533323j
 0.98228725+1.18738131j 0.96858316+1.24868989j], 

orbit: [0.         0.06283185 0.12566371 0.18849556 0.25132741]


### Eigenvalues at each kappa point

In [6]:
# calculating eigenvalues for each kappa on the kappa orbit
eigens = np.vstack([np.linalg.eigvals(H5(kappa)) for kappa in kappas])

# getting real and imaginary parts and stacking them into a tensor
# of shape (l, m, n) to feed into the SGA 
# l: number of points on the orbit
# m: number of eigenvalues
# n: number of eigenvalue features (real and imaginary parts, so n = 2)
eigens_real = eigens.real
eigens_imag = eigens.imag

eigens_for_grouping = np.stack((eigens_real, eigens_imag), axis=-1)
eigens_for_grouping.shape

(100, 5, 2)

### Stepwise-Grouping eigenvalues

In [33]:
from searchep.data import stepwise_grouping

# perform SGA on the eigenvalues
grouped_eigens = stepwise_grouping(eigens_for_grouping)
grouped_eigens_complex = grouped_eigens[..., 0] + 1j*grouped_eigens[..., 1]

In [19]:
norms = np.linalg.norm(grouped_eigens, axis=2, keepdims=True)
norms[norms==0] = 1

In [22]:
normed_grouped_eigens = grouped_eigens / norms
print(grouped_eigens.shape, normed_grouped_eigens.shape)

(100, 5, 2) (100, 5, 2)


In [29]:
initial_set = initial_dataset_old(grouped_eigens, normed_grouped_eigens)

norm dim:  (5, 100, 5)


In [30]:
p_vals_initial = idx_pair[:, 0]
s_vals_initial = idx_pair[:, 1]

In [34]:
from searchep.data import Data
from searchep.gpr import gp_2d_diff_kappa, gp_2d_sum_kappa

data = Data(filename='', directory='', evs_ep=False)
data.kappa = kappas
data.phi = orbit
data.evs = grouped_eigens_complex
data.p = p_vals_initial
data.s = s_vals_initial

PermissionError: [Errno 13] Permission denied: '.'